
This tutorial is advanced and you only need to go through this if you want to know the internals of `Gridap` and what it does under the hood. Even though you will likely want to use the high-level APIs in `Gridap`, this tutorial will (hopefully) help if you want to become a `Gridap` developer, not just a user. We also consider that this tutorial shows how powerful and expressive the `Gridap` kernel is, and how mastering it you can implement new algorithms not currently provided by the library.

As any other Gridap tutorial, this tutorial is primarily designed to be executed in a Jupyter notebook environment. However, the usage of a Julia debugger (typically outside of a Jupyter notebook environment), such as, e.g., the Julia REPL-based [`Debugger.jl`](https://github.com/JuliaDebug/Debugger.jl) package, or the one which comes along with the Visual Studio Code (VSCode) extension for the Julia programming language, may help the reader eager to understand the full detail of the explanations given. Some of the observations that come along with the code snippets are quite subtle/technical and may require a deeper exploration of the underlying code using a debugger.

Let us start including `Gridap` and some of its submodules, to have access to a rich set of not so high-level methods. Note that the module `Gridap` provides the high-level API, whereas the submodules such as, e.g., `Gridap.FESpaces`, provide access to the different parts of the low-level API.



In [66]:

using Gridap
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils

## Discrete model and FE spaces set up using high-level API

We first create the geometry model and FE spaces using the high-level API. In this tutorial, we are not going to describe the geometrical machinery in detail, only what is relevant for the discussion. To simplify the analysis of the outputs, you can consider a 2D mesh, i.e., `D=2` (everything below works for any spatial dimension without any extra complication). In order to make things slightly more interesting, i.e., having non-constant Jacobians, we have considered a mesh that is a stretching of an equal-sized structured mesh.


In [67]:
L = 2 # Domain length in each space dimension
D = 2 # Number of spatial dimensions
n = 4 # Partition (i.e., number of cells per space dimension)

function stretching(x::Point)
   m = zeros(length(x))
   m[1] = x[1]^2
   for i in 2:D
     m[i] = x[i]
   end
   Point(m)
end

pmin = Point(Fill(0,D))
pmax = Point(Fill(L,D))
partition = Tuple(Fill(n,D))
model = CartesianDiscreteModel(pmin,pmax,partition,map=stretching)

CartesianDiscreteModel()

The next step is to build the global FE space of functions from which we are going to extract the unknown function of the differential problem at hand. This tutorial explores the Galerkin discretization of the scalar Poisson equation. Thus, we need to build H1-conforming global FE spaces. This can be achieved using $C^0$ continuous functions made of piece(cell)-wise polynomials. This is precisely the purpose of the following lines of code.

First, we build a scalar-valued (`T = Float64`) Lagrangian reference FE of order `order` atop a reference n-cube of dimension `D`. To this end, we first need to create a `Polytope` using an array of dimension `D` with the parameter `HEX_AXIS`, which encodes the reference representation of the cells in the mesh. Then, we create the Lagrangian reference FE using the reference geometry just created in the previous step. It is not the purpose of this tutorial to describe the (key) abstract concept of `ReferenceFE` in Gridap.


In [68]:
T = Float64
order = 1
pol = Polytope(Fill(HEX_AXIS,D)...) # Quad element
reffe = LagrangianRefFE(T,pol,order)
@which LagrangianRefFE(T, pol, order)
# ref_fe = LagrangianRefFE(Float64, QUAD, 1)
get_node_coordinates(reffe::LagrangianRefFE) 
get_polytope(reffe::LagrangianRefFE)




QUAD

Second, we build the test (Vₕ) and trial (Uₕ) global finite element (FE) spaces out of `model` and `reffe`. At this point we also specify the notion of conformity that we are willing to satisfy, i.e., H1-conformity, and the region of the domain in which we want to (strongly) impose Dirichlet boundary conditions, the whole boundary of the box in this case.


In [69]:
Vₕ = FESpace(model,reffe;conformity=:H1,dirichlet_tags="boundary")
u(x) = x[1]            # Analytical solution (for Dirichlet data)
Uₕ = TrialFESpace(Vₕ,u)


TrialFESpace()

We also want to extract the triangulation out of the model and create a numerical quadrature. We use a quadrature rule with a higher number integration points than those strictly needed to integrate a mass matrix exactly, i.e., `4*order`, instead of `2*order` We do so in order to help the reader distinguish the axis used for quadrature points, and the one used for DoFs in multi-dimensional arrays, which contain the result of evaluating fields (or a differential operator acting on these) in a set of quadrature rule evaluation points.


In [70]:
Tₕ = Triangulation(model)
Qₕ = CellQuadrature(Tₕ,2*order)

CellQuadrature()

`CellDatum` is the root of one out of three main type hierarchies in Gridap (along with the ones rooted at the abstract types `Map` and `Field`) on which the evaluation of variational methods in finite-dimensional spaces is grounded on. Any developer of Gridap should familiarize with these three hierarchies to some extent. Along this tutorial we will give some insight on the rationale underlying these, with some examples, but more effort in the form of self-research is expected from the reader as well.

Conceptually, an instance of a `CellDatum` represents a collection of quantities (e.g., points in a reference system, or scalar-, vector- or tensor-valued fields, or arrays made of these), once per each cell of a triangulation. Using the `get_data` generic function one can extract an array with such quantities. For example, in the case of Qₕ, we get an array of quadrature rules for numerical integration.


In [71]:
Qₕ_cell_data = get_data(Qₕ)
#
# @test length(Qₕ_cell_data) == num_cells(Tₕ)

16-element Fill{GenericQuadrature{2, Float64, Vector{VectorValue{2, Float64}}, Vector{Float64}}}, with entries equal to GenericQuadrature()

In this case we get the same quadrature rule in all cells (note that the returned array is of type `Fill`). Gridap also supports different quadrature rules to be used in different cells. Exploring such feature is out of scope of the present tutorial.

Any `CellDatum` has a trait, the so-called `DomainStyle` trait. This information is consumed by `Gridap` in different parts of the code. It specifies whether the quantities in it are either expressed in the reference (`ReferenceDomain`) or the physical (`PhysicalDomain`) domain. We can indeed check the `DomainStyle` of a `CellDatum` using the `DomainStyle` generic function:


In [72]:
DomainStyle(Qₕ) == ReferenceDomain()
#
DomainStyle(Qₕ) == PhysicalDomain()

false

If we evaluate the two expressions above, we can see that the `DomainStyle` trait of Qₕ is `ReferenceDomain`. This means that the local FE space in the physical space in which our problem is posed is expressed in terms of the composition of a space in a reference FE in a parametric space (which is being shared by many or all FEs in the physical space) and the inverse of the geometrical map (from the parametric to the physical space).

In practise, the integration in the physical space is transformed into a numerical integration in the reference space (via a change of variables) using a quadrature. We can exploit this property for `ReferenceDomain` FE spaces to reduce computations, i.e., to avoid applying the geometrical map to the quadrature points within Qₕ and its inverse at the shape functions in the physical space.

We note that, while finite elements may not be defined in this parametric space (it is though standard practice with Lagrangian FEs, and other FEs, because of performance reasons), finite element functions are always integrated in such a parametric space. However, for FE spaces that are genuinely defined in the physical space, i.e., the ones with the `PhysicalDomain` trait, the transformation of quadrature points from the reference to the physical space is required.

In fact, the `DomainStyle` metadata of `CellDatum` allows `Gridap` to do the right thing (as soon as it is implemented) for all combinations of points and FE spaces (both either expressed in the reference or physical space). This is accomplished by the `change_domain` function in the API of `CellDatum`.

Using the array of quadrature rules `Qₕ_cell_data`, we can access specific entries. The object retrieved provides an array of points (`Point` data type in `Gridap`) in the cell reference parametric space $[0,1]^d$ and their corresponding weights.


In [73]:
q = Qₕ_cell_data[rand(1:num_cells(Tₕ))]
#
p = get_coordinates(q)
#
# w = get_weights(q)


4-element Vector{VectorValue{2, Float64}}:
 VectorValue{2, Float64}(0.21132486540518708, 0.21132486540518708)
  VectorValue{2, Float64}(0.7886751345948129, 0.21132486540518708)
  VectorValue{2, Float64}(0.21132486540518708, 0.7886751345948129)
   VectorValue{2, Float64}(0.7886751345948129, 0.7886751345948129)

However, there is a more convenient way (for reasons made clear above) to work with the evaluation points of quadratures rules in Gridap. Namely, using the get_cell_points function we can extract a CellPoint object out of a CellQuadrature.

In [74]:
Qₕ_cell_point = get_cell_points(Qₕ)
qₖ = get_data(Qₕ_cell_point)

16-element Fill{Vector{VectorValue{2, Float64}}}, with entries equal to VectorValue{2, Float64}[(0.21132486540518708, 0.21132486540518708), (0.7886751345948129, 0.21132486540518708), (0.21132486540518708, 0.7886751345948129), (0.7886751345948129, 0.7886751345948129)]

Not surprisingly, the DomainStyle trait of the CellPoint object is ReferenceDomain, and we get a (cell) array with an array of Points per each cell out of a CellPoint. As seen in the sequel, CellPoints are relevant objects because they are the ones that one can use in order to evaluate the so-called CellField objects on the set of points of a CellPoint.

CellField is an abstract type rooted at a hierarchy that plays a cornerstone role in the implementation of the finite element method in Gridap. At this point, the reader should keep in mind that the finite element method works with global spaces of functions which are defined piece-wise on each cell of the triangulation. In a nutshell (more in the sections below), a CellField, as it being a subtype of CellDatum, might be understood as a collection of Fields (or arrays made out them) per each triangulation cell. Field represents a field, e.g., a scalar, vector, or tensor field. Thus, the domain of a Field are points in the physical domain (represented by a type Point in Gridap, which is a VectorValue with a dimension matching that of the environment space) and the range is a scalar, vector (represented by VectorValue) or tensor (represented by TensorValue).

Unlike a plain array of Fields, a CellField is associated to a triangulation and is specifically designed having in mind FEs. For example, a global finite element function, or the collection of shape basis functions in the local FE space of each cell are examples of CellField objects. As commented above, these fields can be defined in the physical or a reference space (combined with a geometrical map provided by the triangulation object for each cell). Thus, CellField (as a sub-type of CellDatum) has the DomainStyle metadata that is used, e.g., for point-wise evaluations (as indicated above) of the fields and their derivatives (by implementing the transformations when taking a differential operators, e.g., the pull-back of the gradients).

### Exploring our first CellField objects
Let us work with our first CellField objects, namely FEBasis objects, and its evaluation. In particular, let us extract out of the global test space, Vₕ, and trial space, Uₕ, a collection of local test and trial finite element shape basis functions, respectively.

In [75]:
dv = get_fe_basis(Vₕ)
du = get_trial_fe_basis(Uₕ)
grad_dv = ∇(dv)
@test isa(grad_dv, Gridap.FESpaces.FEBasis)
grad_dv_array = get_data(grad_dv)
aa=evaluate(grad_dv,Qₕ_cell_point)
dv_at_Qₕ = evaluate(dv,Qₕ_cell_point)
du_at_Qₕ = evaluate(du,Qₕ_cell_point)



16-element Fill{Gridap.Fields.TransposeFieldIndices{Matrix{Float64}, Float64}}, with entries equal to [0.6220084679281461; 0.16666666666666669; 0.16666666666666663; 0.044658198738520505;;; 0.16666666666666663; 0.6220084679281462; 0.04465819873852045; 0.16666666666666663;;; 0.16666666666666663; 0.04465819873852045; 0.6220084679281462; 0.16666666666666663;;; 0.044658198738520435; 0.16666666666666663; 0.16666666666666663; 0.6220084679281462]

In [76]:
shape_test  = evaluate(dv,Qₕ_cell_point)[1]
du_at_Qₕ = evaluate(du,Qₕ_cell_point)[1]

shape_trial = reshape(du_at_Qₕ, 9, 4)

w = ones(9)  # Quadrature weights (simplified; replace with actual weights)
detJK = 1.0  # Jacobian determinant (simplified; depends on element)
num_test_functions = 4
num_trial_functions = 4
num_quadrature_points = 9
M = zeros(4,4)
# Loop-based computation
for i in 1:num_quadrature_points
    detJK_wi = detJK * w[i]
    for j in 1:num_test_functions
        for k in 1:num_trial_functions
            M[j,k] += shape_test[i,j] * shape_trial[i,k] * detJK_wi
        end
    end
end
M

DimensionMismatch: DimensionMismatch: parent has 16 elements, which is incompatible with size (9, 4)

In [77]:
dv_at_Qₕ[rand(1:num_cells(Tₕ))]
M[:,:] = 0.0
for i in 1:num_quadrature_points
  detJK_wi = det(JK) * w[i]
  for j in 1:num_test_functions
    for k in 1:num_trial_functions
      M[j,k] += shape_test[i,j] * shape_trial[i,k] * detJK_wi
    end
  end
end

ArgumentError: ArgumentError: indexed assignment with a single value to possibly many locations is not supported; perhaps use broadcasting `.=` instead?

In [78]:
dv_mult_du = du*dv

OperationCellField():
 num_cells: 16
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10857374271509949750

In [79]:
dv_mult_du_at_Qₕ = evaluate(dv_mult_du,Qₕ_cell_point)


16-element Fill{Array{Float64, 3}}, with entries equal to [0.38689453417431957 0.10366807798802433 0.10366807798802433 0.027777777777777762; 0.027777777777777783 0.10366807798802438 0.007443033123086742 0.027777777777777776; 0.027777777777777766 0.0074430331230867395 0.10366807798802435 0.027777777777777766; 0.0019943547145691944 0.007443033123086749 0.007443033123086749 0.02777777777777781;;; 0.10366807798802433 0.027777777777777766 0.027777777777777766 0.007443033123086738; 0.10366807798802438 0.38689453417431974 0.027777777777777776 0.10366807798802435; 0.0074430331230867395 0.0019943547145691892 0.027777777777777776 0.0074430331230867395; 0.007443033123086749 0.027777777777777766 0.027777777777777766 0.10366807798802435;;; 0.10366807798802433 0.027777777777777766 0.027777777777777766 0.007443033123086738; 0.007443033123086742 0.027777777777777776 0.0019943547145691892 0.0074430331230867395; 0.10366807798802435 0.027777777777777776 0.38689453417431974 0.10366807798802435; 0.00744303

In [80]:
m=Broadcasting(*)

Broadcasting()

In [81]:
A=evaluate(m,dv_at_Qₕ[rand(1:num_cells(Tₕ))],du_at_Qₕ[rand(1:num_cells(Tₕ))])


4×4 Matrix{Float64}:
 0.103668    0.0277778   0.0277778   0.00744303
 0.0277778   0.103668    0.00744303  0.0277778
 0.0277778   0.00744303  0.103668    0.0277778
 0.00744303  0.0277778   0.0277778   0.103668

In [82]:
B=broadcast(*,dv_at_Qₕ[rand(1:num_cells(Tₕ))],du_at_Qₕ[rand(1:num_cells(Tₕ))])


4×4 Matrix{Float64}:
 0.0277778   0.00744303  0.00744303  0.00199435
 0.00744303  0.0277778   0.00199435  0.00744303
 0.00744303  0.00199435  0.0277778   0.00744303
 0.00199435  0.00744303  0.00744303  0.0277778

In [83]:
dv_array = get_data(dv)
ϕ₃ = dv_array[1][3]
evaluate(ϕ₃,[Point(0,0),Point(1,0),Point(0,1),Point(1,1)])



4-element Vector{Float64}:
 0.0
 0.0
 1.0
 0.0

In [84]:
ϕ = dv_array[1]
evaluate(ϕ,[Point(0,0),Point(1,0),Point(0,1),Point(1,1)])

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [86]:
dv_array = get_data(dv)
dv_array_at_qₖ = lazy_map(evaluate,dv_array,qₖ)
dv_array_at_qₖ[1]
qₖ

16-element Fill{Vector{VectorValue{2, Float64}}}, with entries equal to VectorValue{2, Float64}[(0.21132486540518708, 0.21132486540518708), (0.7886751345948129, 0.21132486540518708), (0.21132486540518708, 0.7886751345948129), (0.7886751345948129, 0.7886751345948129)]

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*